## Preprocessing our data

In [42]:
#Function to treat outliers
def outlier_treatment(datacolumn):
        sorted(datacolumn)
        Q1,Q3 = np.percentile(datacolumn , [25,75])
        IQR = Q3 - Q1
        lower_range = Q1 -(1.5 * IQR)
        upper_range = Q3 + (1.5 * IQR)
        return lower_range,upper_range

In [43]:
for i in outlier_col.columns:
    lower_range,upper_range = outlier_treatment(data_df[i])
    data_df.drop(data_df[ (data_df[i] > upper_range) | (data_df[i] < lower_range) ].index , inplace=True)

In [44]:
outliers_count_df = pd.DataFrame(columns=['Outlier_Count','skewness'])
for column in data_df[outlier_col.columns]:
    if pd.api.types.is_numeric_dtype(data_df[column]):
        Q1 = data_df[column].quantile(0.25)
        Q3 = data_df[column].quantile(0.75)
        IQR = Q3-Q1
        lower_whisker = Q1-1.5*IQR
        upper_whisker = Q3+1.5*IQR
        outliers_count_df.loc[column] = [((data_df[column]<lower_whisker) | (data_df[column] > upper_whisker)).sum(),numeric_col[column].skew()]
outliers_count_df

,Outlier_Count,skewness
age,0.0,0.784697
campaign,0.0,4.762507
pdays,0.0,7.939537
previous,0.0,3.832042
cons.conf.idx,0.0,0.303180


In [45]:
#standardizing with sklearn
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

for i in numeric_col:
    data_df[i] = scale.fit_transform(data_df[[i]])
data_df.describe()

,age,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
count,3.276500e+04,3.276500e+04,32765.0,32765.0,3.276500e+04,3.276500e+04,3.276500e+04,3.276500e+04,3.276500e+04,32765.000000
mean,3.539160e-16,3.122788e-17,0.0,0.0,2.775812e-16,1.554455e-15,6.661948e-16,-1.665487e-16,3.442007e-15,0.086861
std,1.000015e+00,1.000015e+00,0.0,0.0,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,0.281636
min,-2.283344e+00,-8.343910e-01,0.0,0.0,-2.673646e+00,-2.690222e+00,-2.490007e+00,-2.140839e+00,-3.589122e+00,0.000000
25%,-8.063664e-01,-8.343910e-01,0.0,0.0,-3.442465e-01,-8.320847e-01,-5.430842e-01,3.399360e-02,1.504897e-01,0.000000
50%,-1.733760e-01,-6.504187e-02,0.0,0.0,5.028079e-01,5.033932e-01,-3.267594e-01,5.720834e-01,2.294261e-01,0.000000
75%,7.761094e-01,7.043072e-01,0.0,0.0,7.145715e-01,6.447530e-01,9.711891e-01,6.382209e-01,7.606022e-01,0.000000
max,3.097074e+00,3.012354e+00,0.0,0.0,7.145715e-01,2.082531e+00,2.557571e+00,6.915162e-01,7.606022e-01,1.000000


In [46]:
#having all the basic categories in one sub column of basic
data_df.education = data_df.education.replace({"basic.4y":"basic",
                                              "basic.6y":"basic",
                                               "basic.9y":"basic"})

In [47]:
data_df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,1.725595,housemaid,married,basic,no,no,no,telephone,may,mon,-0.834391,0.0,0.0,nonexistent,0.502808,0.644753,0.971189,0.570799,0.15049,0
1,1.831093,services,married,high.school,unknown,no,no,telephone,may,mon,-0.834391,0.0,0.0,nonexistent,0.502808,0.644753,0.971189,0.570799,0.15049,0
2,-0.278874,services,married,high.school,no,yes,no,telephone,may,mon,-0.834391,0.0,0.0,nonexistent,0.502808,0.644753,0.971189,0.570799,0.15049,0
3,0.037621,admin.,married,basic,no,no,no,telephone,may,mon,-0.834391,0.0,0.0,nonexistent,0.502808,0.644753,0.971189,0.570799,0.15049,0
4,1.725595,services,married,high.school,no,no,yes,telephone,may,mon,-0.834391,0.0,0.0,nonexistent,0.502808,0.644753,0.971189,0.570799,0.15049,0


In [48]:
#encoding the data of the categorical columns

#OneHotEncoder is used on normal categories i.e they are just categories
data_encoded = pd.get_dummies(data_df,columns = categorical_col.drop('y',axis=1).columns)
data_encoded.head()

,age,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,...,month_may,month_nov,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_nonexistent
0,1.725595,-0.834391,0.0,0.0,0.502808,0.644753,0.971189,0.570799,0.15049,0,...,1,0,0,0,0,1,0,0,0,1
1,1.831093,-0.834391,0.0,0.0,0.502808,0.644753,0.971189,0.570799,0.15049,0,...,1,0,0,0,0,1,0,0,0,1
2,-0.278874,-0.834391,0.0,0.0,0.502808,0.644753,0.971189,0.570799,0.15049,0,...,1,0,0,0,0,1,0,0,0,1
3,0.037621,-0.834391,0.0,0.0,0.502808,0.644753,0.971189,0.570799,0.15049,0,...,1,0,0,0,0,1,0,0,0,1
4,1.725595,-0.834391,0.0,0.0,0.502808,0.644753,0.971189,0.570799,0.15049,0,...,1,0,0,0,0,1,0,0,0,1


In [49]:
data_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32765 entries, 0 to 41186
Data columns (total 59 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   age                            32765 non-null  float64
 1   campaign                       32765 non-null  float64
 2   pdays                          32765 non-null  float64
 3   previous                       32765 non-null  float64
 4   emp.var.rate                   32765 non-null  float64
 5   cons.price.idx                 32765 non-null  float64
 6   cons.conf.idx                  32765 non-null  float64
 7   euribor3m                      32765 non-null  float64
 8   nr.employed                    32765 non-null  float64
 9   y                              32765 non-null  int64  
 10  job_admin.                     32765 non-null  uint8  
 11  job_blue-collar                32765 non-null  uint8  
 12  job_entrepreneur               32765 non-null 

In [50]:
!pip install imblearn

#SOlving the imbalance in the target column
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=0)
train = data_encoded.drop('y',axis=1)
target = data_encoded['y']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train,target, test_size=0.1, random_state=42)
columns = X_train.columns

smote_train,smote_target=smote.fit_sample(X_train, y_train)
smote_train = pd.DataFrame(data=smote_train,columns=columns )
smote_target= pd.DataFrame(data=smote_target,columns=['y'])
# we can Check the numbers of our data
print("length of oversampled data is ",len(smote_train))
print("Number of no subscription in oversampled data",len(smote_target[smote_target['y']==0]))
print("Number of subscription",len(smote_target[smote_target['y']==1]))
print("Proportion of yes subscription data in oversampled data is ",len(smote_target[smote_target['y']==1])/len(smote_target))
print("Proportion of subscription data in oversampled data is ",len(smote_target[smote_target['y']==1])/len(smote_target))

length of oversampled data is  53852
Number of no subscription in oversampled data 26926
Number of subscription 26926
Proportion of yes subscription data in oversampled data is  0.5
Proportion of subscription data in oversampled data is  0.5
